Universität Heidelberg
Distributed Systems I (IVS1)
Winter Semester 18/19

- Duc Anh Phi
- Michael Tabachnik
- Edgar Brotzmann

# Solutions to Problem Set 4 for lecture Distributed Systems I (IVS1)
## Due: 20.11.2018, 2pm


### Exercise 1

In [15]:
import gzip
import findspark
import re

findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Solutions to Problem Set 4 IVS1") \
    .config("spark.executor.memory", "1gb") \
    .getOrCreate()

sc = spark.sparkContext

In [32]:
def createDataframe(filename):
    with gzip.open(filename, 'rt') as f:
        # use header as row schema
        headerLine = f.readline()
        header = re.split(r'\t+', headerLine.rstrip('\t\n'))[1:]
        RowSchema = Row(*header)
        content = [row for row in f.read().split("\n") if row]
        df = sc.parallelize(content) \
            .map(lambda line: re.split(r'\t+', line.rstrip('\t\n'))[1:]) \
            .map(lambda line: RowSchema(*line)) \
            .toDF()
        
        return df

def convertColumns(df):
    df = df.withColumn('Price', df['Price'].cast(DoubleType()))
    df = df.withColumn('Timestamp', unix_timestamp('Timestamp', "yyyy-MM-dd'T'HH:mm:ssZZZZ"))
    return df

df = createDataframe("prices-ap-northeast-2-2017-11-17.txt.gz")
df = convertColumns(df)
df.groupBy('InstanceType', 'ProductDescription').avg('Price').show()

+------------+------------------+-------------------+
|InstanceType|ProductDescription|         avg(Price)|
+------------+------------------+-------------------+
|  r3.2xlarge|        Linux/UNIX| 0.6557668832731297|
|   c4.xlarge|           Windows|0.22131742268041452|
|   i3.xlarge|           Windows|0.24962053470769482|
|  p2.8xlarge|        Linux/UNIX| 117.20000000000049|
| x1.16xlarge|           Windows|  9.048597719044308|
|  i2.4xlarge|           Windows| 1.5526279912024583|
|    c4.large|           Windows|0.10950416912487713|
|  i3.8xlarge|        Linux/UNIX| 29.279999999999934|
|  r4.2xlarge|        Linux/UNIX|0.21761773442050408|
|  c4.2xlarge|           Windows|   0.42304020895996|
|  d2.4xlarge|           Windows| 1.1192076448974577|
| i3.16xlarge|        Linux/UNIX|   58.5599999999999|
|  r3.4xlarge|        Linux/UNIX| 0.5058647696038822|
|  m4.4xlarge|           Windows| 1.0006524359704119|
|  r4.2xlarge|           Windows| 0.7127052461538115|
|  i3.8xlarge|           Win